In [ ]:
import json

import napari
import pandas as pd
from aicsimageio import AICSImage
from dask_image.imread import imread
from napari_animation import Animation

from fucciphase import process_trackmate
from fucciphase.napari import add_trackmate_data_to_viewer
from fucciphase.phase import estimate_percentage_by_subsequence_alignment
from fucciphase.sensor import FUCCISASensor
from fucciphase.utils import (
    compute_motility_parameters,
)

In [ ]:
track_file = "merged_linked.ome.xml"
cyan_channel_id = "MEAN_INTENSITY_CH2"
magenta_channel_id = "MEAN_INTENSITY_CH1"

In [ ]:
with open("fuccisa_hacat.json") as fp:
    sensor_properties = json.load(fp)
sensor = FUCCISASensor(**sensor_properties)
reference_track = pd.read_csv("hacat_fucciphase_reference.csv")

In [ ]:
track_df = process_trackmate(
    track_file,
    channels=[cyan_channel_id, magenta_channel_id],
    sensor=sensor,
    thresholds=[0.1, 0.1],
    use_moving_average=False,
    generate_unique_tracks=True,
)

In [ ]:
reference_track.rename(
    columns={"cyan": cyan_channel_id, "magenta": magenta_channel_id}, inplace=True
)

In [ ]:
estimate_percentage_by_subsequence_alignment(
    track_df,
    dt=0.25,
    channels=[cyan_channel_id, magenta_channel_id],
    reference_data=reference_track,
    track_id_name="UNIQUE_TRACK_ID",
)

In [ ]:
compute_motility_parameters(track_df, track_id_name="UNIQUE_TRACK_ID")
minutes_per_frame = 15

track_df["VELOCITIES"] /= minutes_per_frame

In [ ]:
track_ids = track_df["UNIQUE_TRACK_ID"].unique()

In [ ]:
labels = imread("dapieq_labels_manual_relabel.tif")

In [ ]:
viewer = napari.Viewer()

In [ ]:
image = AICSImage("6_DenoisedAI_LRDec50_Tub_FullVideo.nd2")
scale = (image.physical_pixel_sizes.Y, image.physical_pixel_sizes.X)

In [ ]:
cyan = image.get_image_dask_data("TYX", C=3)
magenta = image.get_image_dask_data("TYX", C=0)
actin = image.get_image_dask_data("TYX", C=1)

In [ ]:
add_trackmate_data_to_viewer(
    track_df,
    viewer,
    scale=scale,
    image_data=[cyan, magenta],
    colormaps=["cyan", "magenta"],
    labels=labels,
    cycle_percentage_id="CELL_CYCLE_PERC_DTW",
    textkwargs={"size": 14},
)

In [ ]:
viewer.add_image(actin, name="actin", colormap="gray", scale=scale, blending="additive")

# Adjust size according to last frame

In [ ]:
viewer.dims.current_step = (labels.shape[0] - 1, 0, 0)
viewer.reset_view()

# Fix view manually
* Adjust the layers to be viewed
* Move the window until it looks good

In [ ]:
animation = Animation(viewer)
# start animation on first frame
viewer.dims.current_step = (0, 0, 0)
animation.capture_keyframe()
# last frame
viewer.dims.current_step = (labels.shape[0] - 1, 0, 0)
animation.capture_keyframe(steps=labels.shape[0] - 1)
animation.animate(
    "Hacat_percentages_white_similarity_dtw_fucciphase_all_tracks.mov",
    canvas_only=True,
    fps=4,
    quality=9,
    scale_factor=1.0,
)